### Введение в TensorFlow и Keras

CUDA is NVIDIA’s language/API for programming on the graphics card (to write really high performance programs run on the GPU). [cuDNN](https://developer.nvidia.com/cudnn) is a library for deep neural nets built using CUDA. It provides GPU accelerated functionality for common operations in deep neural nets. You could use it directly yourself, but other libraries like TensorFlow already have built abstractions backed by cuDNN.

[Install CUDA 10.0, cuDNN 7.3 and build TensorFlow (GPU) from source on Ubuntu 18.04](https://medium.com/@vitali.usau/install-cuda-10-0-cudnn-7-3-and-build-tensorflow-gpu-from-source-on-ubuntu-18-04-3daf720b83fe)

In [1]:
import tensorflow as tf

/home/xenakas/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
w = tf.Variable(tf.random_normal([3, 2], mean=0.0, stddev=0.4), name= 'weights' )
b = tf.Variable(tf.zeros([2]), name= 'biases' )

In [3]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 7775143218817196035, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 4272953429152955877
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 14225393877924459717
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 5982175232
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 14514309632458873213
 physical_device_desc: "device: 0, name: GeForce GTX 1060 with Max-Q Design, pci bus id: 0000:01:00.0, compute capability: 6.1"]

In [4]:
tf.test.is_gpu_available() 

True

[cuDNN Installation Guide](https://docs.nvidia.com/deeplearning/sdk/cudnn-install/index.html#troubleshoot)

In [5]:
tf.test.is_gpu_available(
    cuda_only=False,
    min_cuda_compute_capability=None
)

True

In [6]:
tf.test.gpu_device_name()

'/device:GPU:0'

[gcc-7 for CUDA](https://stackoverflow.com/questions/53344283/gcc-versions-later-than-7-are-not-supported-by-cuda-10-qt-error-in-arch-linux/53828864#53828864)

[Build tf from source with bazel](https://www.tensorflow.org/install/source)

Для переменных можно явным образом указать, где именно им нужно находить-
ся в памяти, впрочем, для TensorFlow-GPU все тензоры
по умолчанию будут инициализироваться на вашей видеокарте.

In [9]:
with tf.device('/device:GPU:0'):
    w = tf.Variable(tf.random_normal([3, 2], mean=0.0, stddev=0.4), name= 'weights' )

In [10]:
w, b

(<tf.Variable 'weights_1:0' shape=(3, 2) dtype=float32_ref>,
 <tf.Variable 'biases:0' shape=(2,) dtype=float32_ref>)

Все переменные обязательно нужно инициализировать:

In [13]:
init = tf.global_variables_initializer()

Но это не работает в тех случаях, когда нужно инициализировать переменные
из значений других переменных

In [35]:
w2 = tf.Variable(w.initialized_value(), name= 'w2' ) #depreciated
w2

<tf.Variable 'w2_5:0' shape=(3, 2) dtype=float32_ref>

In [36]:
w3 = w.read_value() 
w3

<tf.Tensor 'read_6:0' shape=(3, 2) dtype=float32>

In [37]:
w4 = tf.Variable(w.read_value()) 
w4

<tf.Variable 'Variable_2:0' shape=(3, 2) dtype=float32_ref>

Все переменные текущей сессии можно в любой момент сохранить в файл:

```python
# Create a saver.
saver = tf.train.Saver(...variables...)

# Launch the graph and train, saving the model every 1,000 steps.
sess = tf.Session()
for step in xrange(1000000):
    sess.run(..training_op..)
    if step % 1000 == 0:
        # Append the step number to the checkpoint name:
        saver.save(sess, 'model.ckpt', global_step=step)
```

Эта процедура сохранит все переменные сессии `sess ` в файл `model.ckpt` , а чтобы
потом восстановить сессию, достаточно запустить:

```python
saver.restore( 'model.ckpt' )
```

Для удобства передачи новых данных на видео-
карту в TensorFlow существуют специальные тензоры — так называемые заглушки,
`tf.placeholder `, которым нужно сначала передать только тип данных и размерности
тензора, а затем сами данные будут подставлены уже в момент вычислений:

In [43]:
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)

output = tf.multiply(x, y)

with tf.Session() as sess:
    result = sess.run(output, feed_dict={x: 2, y: 3})

In [44]:
result

6.0

#### Broadcasting

Broadcasting применим для всех поэлементных операций над двумя тензорами. Размерности двух тензоров последовательно сравниваются начиная с конца; при каждом
сравнении необходимо выполнение одного из двух условий:

- либо размерности равны;
- либо одна из размерностей равна 1.

При этом тензоры не обязаны иметь одинаковую размерность: недостающие измерения меньшего из тензоров будут интерпретированы как единичные. 

Размерность получаемого на выходе тензора, если все условия выполнены, вычисляется как максимум из соответствующих размерностей исходных тензоров. 

К каждому столбцу матрицы поэлементно добавить один
и тот же вектор (m — это матрица размера 10 × 100, а v — вектор длины 100, и при сложении
v будет прибавлен к каждому столбцу m)

In [46]:
m = tf.Variable(tf.random_normal([10, 100], mean=0.0, stddev=0.4), name= 'matrix' )
v = tf.Variable(tf.random_normal([100], mean=0.0, stddev=0.4), name= 'vector' )

result = m + v

In [58]:
sess = tf.Session()

init = tf.global_variables_initializer()

sess.run(init)

print(sess.run([result]))

[array([[ 6.04309201e-01,  2.55050898e-01, -7.01737642e-01,
         1.56931496e+00, -1.75524309e-01,  6.53161645e-01,
        -4.62540925e-01,  1.70202947e+00, -4.31098163e-01,
         4.94152427e-01, -1.16099608e+00,  1.13741231e+00,
        -5.09247065e-01, -5.48830986e-01, -2.35134408e-01,
         5.72927296e-01,  4.07880962e-01, -7.63735771e-02,
        -4.67362285e-01,  4.40888911e-01,  1.54728323e-01,
         4.56663251e-01, -6.72159493e-01, -2.46533707e-01,
         5.67557812e-01, -9.24277782e-01,  2.37291962e-01,
         6.65069640e-01,  1.12259835e-01, -5.26782691e-01,
        -2.12706521e-01, -3.55387807e-01, -5.70140839e-01,
         1.13313675e-01,  6.42575324e-01, -1.42393753e-01,
        -7.61145353e-01,  4.93229121e-01,  1.22595942e+00,
         3.60473394e-01, -6.26312315e-01, -1.73497653e+00,
         5.14786243e-01,  7.12987781e-02,  1.14329562e-01,
        -1.68814659e-02, -2.32101455e-01,  4.56009567e-01,
        -5.81486702e-01, -1.02432430e+00, -4.34047461e-

In [53]:
tf.print(result)

<tf.Operation 'PrintV2_3' type=PrintV2>

In [52]:
sess = tf.Session()
with sess.as_default():
    tensor = tf.range(10)
    print_op = tf.print(tensor)
    with tf.control_dependencies([print_op]):
        out = tf.add(tensor, tensor)
sess.run(out)
    

array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18], dtype=int32)

Среднее будет вычисляться по второй размерности тензора tensor , то
есть мы десять раз усредним по 100 чисел и получим вектор длины 10.

In [60]:
tensor = tf.placeholder(tf.float32, [10, 100])
result = tf.reduce_mean(tensor, axis=1)
result

<tf.Tensor 'Mean_1:0' shape=(10,) dtype=float32>

#### Пространства переменных (variable scopes)

Этот механизм состоит из двух основных функций:
- `tf.get_variable(<name>, <shape>, <initializer>)` создает или возвращает переменную с заданным именем
- `tf.variable_scope(<scope_name>)` управляет пространствами имен, использующихся в `tf.get_variable()`.

In [61]:
def linear_transform(vec, shape):
    with tf.variable_scope( 'transform' ):
        w = tf.get_variable( 'matrix', shape, initializer=tf.random_normal_initializer())
        return tf.matmul(vec, w)

In [81]:
import sys
sys.maxsize

9223372036854775807

In [89]:
n = tf.Variable(tf.random.uniform([10,10], minval=0, maxval=10), name= 'ppois' )

with tf.variable_scope( 'linear_transformers' ) as scope:
    result1 = linear_transform(m, (100,10))
    scope.reuse_variables()
    result2 = linear_transform(n, (10,10))

In [90]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
print(sess.run([result2]))
print(sess.run([n]))

[array([[-13.465744  , -10.51856   ,   0.6307935 ,  -3.3713117 ,
         -0.23475742,   5.7364316 ,  15.951176  , -19.022282  ,
         -4.4012003 ,   2.9477272 ],
       [-16.098442  , -17.740982  ,   8.247864  , -21.681013  ,
          7.6532326 ,  -3.2892995 ,  24.584433  ,  -6.6525908 ,
         -0.58899844, -10.030921  ],
       [-20.109829  , -34.318428  ,   9.91441   , -19.19573   ,
          5.3231835 ,   2.6585448 ,  39.454746  , -32.01182   ,
        -15.228199  ,   0.28263783],
       [-15.86194   , -14.640007  ,   3.3538852 , -13.953714  ,
          2.4311757 ,   2.9946008 ,  15.273175  , -19.917723  ,
         -2.9512804 ,  -0.66445565],
       [-28.142452  , -33.394283  ,   5.5181394 , -10.855414  ,
          4.6550407 , -18.12253   ,  35.613377  , -19.951536  ,
         -6.2578583 ,  -2.1858988 ],
       [ -9.8563595 , -22.050272  ,  10.604767  , -24.21849   ,
          6.439405  ,   4.723533  ,  31.737251  ,  -7.090474  ,
         -6.1590276 , -15.653449  ],
       [-

####  Линейная регрессия

Линейная регрессия —
это фактически один нейрон, который получает на вход вектор значений x.

In [92]:
import numpy as np
n_samples, batch_size, num_steps = 1000, 100, 2000
X_data = np.random.uniform(1, 10, (n_samples, 1))
y_data = 2 * X_data + 1 + np.random.normal(0, 2, (n_samples, 1))

In [93]:
X = tf.placeholder(tf.float32, shape=(batch_size, 1))
y = tf.placeholder(tf.float32, shape=(batch_size, 1))

In [94]:
with tf.variable_scope( 'linear-regression' ):
    k = tf.Variable(tf.random_normal((1, 1)), name= 'slope')
    b = tf.Variable(tf.zeros((1,)), name= 'bias')

y_pred = tf.matmul(X, k) + b
loss = tf.reduce_sum((y - y_pred) ** 2) # сумма матрица по строчкам
optimizer = tf.train.GradientDescentOptimizer(0.01).minimize(loss)

In [96]:
display_step = 100
with tf.Session() as sess:
   
    with tf.variable_scope( 'linear-regression' ):
        k = tf.Variable(tf.random_normal((1, 1)), name= 'slope')
        b = tf.Variable(tf.zeros((1,)), name= 'bias')

    sess.run(tf.global_variables_initializer())
    for i in range(num_steps):
        indices = np.random.choice(n_samples, batch_size)
        X_batch, y_batch = X_data[indices], y_data[indices]
        _, loss_val, k_val, b_val = sess.run([optimizer, loss, k, b], feed_dict = { X : X_batch, y : y_batch })
        
        if (i+1) % display_step == 0: 
            print( 'Эпоха %d: %.8f, k=%.4f, b=%.4f' % (i+1, loss_val, k_val, b_val))
            #print( sess.run([tf.matmul(X, k)], feed_dict = { X : X_batch, y : y_batch }) )

Эпоха 100: nan, k=-0.7014, b=0.0000
Эпоха 200: nan, k=-0.7014, b=0.0000
Эпоха 300: nan, k=-0.7014, b=0.0000
Эпоха 400: nan, k=-0.7014, b=0.0000
Эпоха 500: nan, k=-0.7014, b=0.0000
Эпоха 600: nan, k=-0.7014, b=0.0000
Эпоха 700: nan, k=-0.7014, b=0.0000
Эпоха 800: nan, k=-0.7014, b=0.0000
Эпоха 900: nan, k=-0.7014, b=0.0000
Эпоха 1000: nan, k=-0.7014, b=0.0000
Эпоха 1100: nan, k=-0.7014, b=0.0000
Эпоха 1200: nan, k=-0.7014, b=0.0000
Эпоха 1300: nan, k=-0.7014, b=0.0000
Эпоха 1400: nan, k=-0.7014, b=0.0000
Эпоха 1500: nan, k=-0.7014, b=0.0000
Эпоха 1600: nan, k=-0.7014, b=0.0000
Эпоха 1700: nan, k=-0.7014, b=0.0000
Эпоха 1800: nan, k=-0.7014, b=0.0000
Эпоха 1900: nan, k=-0.7014, b=0.0000
Эпоха 2000: nan, k=-0.7014, b=0.0000


In [13]:
print( 'Эпоха %d: %.8f, k=%.4f, b=%.4f' % (10, 4, 5, 6))

Эпоха 10: 4.00000000, k=5.0000, b=6.0000
